In [1]:
import pandas as pd
import numpy as np
import dask.array as da
import geopandas
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt

# Top twenty MSA
- New York-Newark-Jersey City, NY-NJ-PA MSA 19,768,458 20,140,470 −1.85% New York-Newark, NY-NJ-CT-PA CSA
- Los Angeles-Long Beach-Anaheim, CA MSA 12,997,353 13,200,998 −1.54% Los Angeles-Long Beach, CA CSA
- Chicago-Naperville-Elgin, IL-IN-WI MSA 9,509,934 9,618,502 −1.13% Chicago-Naperville, IL-IN-WI CSA
- Dallas-Fort Worth-Arlington, TX MSA 7,759,615 7,637,387 +1.60% Dallas-Fort Worth, TX-OK CSA
- Houston-The Woodlands-Sugar Land, TX MSA 7,206,841 7,122,240 +1.19% Houston-The Woodlands, TX CSA
- Washington-Arlington-Alexandria, DC-VA-MD-WV MSA 6,356,434 6,385,162 −0.45% Washington-Baltimore-Arlington, DC-MD-VA-WV-PA CSA
- Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA 6,228,601 6,245,051 −0.26% Philadelphia-Reading-Camden, PA-NJ-DE-MD CSA
- Atlanta-Sandy Springs-Alpharetta, GA MSA 6,144,050 6,089,815 +0.89% Atlanta–Athens-Clarke County–Sandy Springs, GA-AL CSA
- Miami-Fort Lauderdale-West Palm Beach, FL MSA 6,091,747 6,138,333 −0.76% Miami-Port St. Lucie-Fort Lauderdale, FL CSA
- Phoenix-Mesa-Chandler, AZ MSA 4,946,145 4,845,832 +2.07% Phoenix-Mesa, AZ CSA
- Boston-Cambridge-Newton, MA-NH MSA 4,899,932 4,941,632 −0.84% Boston-Worcester-Providence, MA-RI-NH-CT CSA
- Riverside-San Bernardino-Ontario, CA MSA 4,653,105 4,599,839 +1.16% Los Angeles-Long Beach, CA CSA
- San Francisco-Oakland-Berkeley, CA MSA 4,623,264 4,749,008 −2.65% San Jose-San Francisco-Oakland, CA CSA
- Detroit–Warren–Dearborn, MI MSA 4,365,205 4,392,041 −0.61% Detroit-Warren-Ann Arbor, MI CSA
- Seattle-Tacoma-Bellevue, WA MSA 4,011,553 4,018,762 −0.18% Seattle-Tacoma, WA CSA
- Minneapolis-St. Paul-Bloomington, MN-WI MSA 3,690,512 3,690,261 +0.01% Minneapolis-St. Paul, MN-WI CSA
- San Diego-Chula Vista-Carlsbad, CA MSA 3,286,069 3,298,634 −0.38% 
- Tampa-St. Petersburg-Clearwater, FL MSA 3,219,514 3,175,275 +1.39% 
- Denver-Aurora-Lakewood, CO MSA 2,972,566 2,963,821 +0.30% Denver-Aurora, CO CSA
- Baltimore-Columbia-Towson, MD MSA 2,838,327 2,844,510 −0.22% Washington-Baltimore-Arlington, DC-MD-VA-WV-PA CSA

In [2]:
# the name doesn't match, re-adjusted after looking at the MSAnamelsad field in Rest_Attribute_Location_2022.sav
top_msa = ['New York-Newark-Jersey City, NY-NJ-PA Metro Area', 
          'Los Angeles-Long Beach-Anaheim, CA Metro Area',
          'Chicago-Naperville-Elgin, IL-IN-WI Metro Area',
          'Dallas-Fort Worth-Arlington, TX Metro Area',
          'Houston-The Woodlands-Sugar Land, TX Metro Area',
          'Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area',
          'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area',
          'Atlanta-Sandy Springs-Alpharetta, GA Metro Area',
          'Miami-Fort Lauderdale-Pompano Beach, FL Metro Area', # discrepency no west in the name
          'Phoenix-Mesa-Chandler, AZ Metro Area',
          'Boston-Cambridge-Newton, MA-NH Metro Area',
          'Riverside-San Bernardino-Ontario, CA Metro Area',
          'San Francisco-Oakland-Berkeley, CA Metro Area',
          'Detroit-Warren-Dearborn, MI Metro Area',
          'Seattle-Tacoma-Bellevue, WA Metro Area',
          'Minneapolis-St. Paul-Bloomington, MN-WI Metro Area',
          'San Diego-Chula Vista-Carlsbad, CA Metro Area',
          'Tampa-St. Petersburg-Clearwater, FL Metro Area',
          'Denver-Aurora-Lakewood, CO Metro Area',
          'Baltimore-Columbia-Towson, MD Metro Area']

In [3]:
folder = 'F:/data/restaurant'
shp_path = folder+'/CBG_border/CBG_Border.shp'

In [4]:
output_path = folder+'/parquet'
output_tmp_path = folder+'/parquet_tmp_msa'
output_tmp_msa_only_path = folder+'/parquet_tmp_msa_only'

In [5]:
df = pd.read_spss(folder+'/Rest_Attribute_Location_2022.sav')

In [ ]:
df.head()

In [7]:
#validate that the top 20 msas exist in the data
for m in top_msa:
    print(df[df['MSAnamelsad']==m].shape[0], m)

22659 New York-Newark-Jersey City, NY-NJ-PA Metro Area
11025 Los Angeles-Long Beach-Anaheim, CA Metro Area
5842 Chicago-Naperville-Elgin, IL-IN-WI Metro Area
3110 Dallas-Fort Worth-Arlington, TX Metro Area
2470 Houston-The Woodlands-Sugar Land, TX Metro Area
4184 Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area
4798 Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area
2742 Atlanta-Sandy Springs-Alpharetta, GA Metro Area
2676 Miami-Fort Lauderdale-Pompano Beach, FL Metro Area
1204 Phoenix-Mesa-Chandler, AZ Metro Area
5244 Boston-Cambridge-Newton, MA-NH Metro Area
2280 Riverside-San Bernardino-Ontario, CA Metro Area
3964 San Francisco-Oakland-Berkeley, CA Metro Area
2595 Detroit-Warren-Dearborn, MI Metro Area
2188 Seattle-Tacoma-Bellevue, WA Metro Area
1291 Minneapolis-St. Paul-Bloomington, MN-WI Metro Area
1860 San Diego-Chula Vista-Carlsbad, CA Metro Area
1436 Tampa-St. Petersburg-Clearwater, FL Metro Area
1386 Denver-Aurora-Lakewood, CO Metro Area
2131 Baltimore-Columbia-Tows

In [8]:
df['place_county'] = df['statefp'] + df['countyfp']
df['place_county'] = df['place_county'].astype(int)
df['is_top20'] = np.where(df['MSAnamelsad'].isin(top_msa), 1, 0)

In [9]:
df_cbg = pd.read_csv(folder+'/Restaurant_CBG.csv')
df_cbg.rename(columns={'CBG_GEOID':'place_cbg'}, inplace=True)
df_cbg = df_cbg[['placekey', 'place_cbg']]

In [10]:
df = df.join(df_cbg.set_index('placekey'), on='Placekey')

In [ ]:
df.head()

In [12]:
df_select = df[['Placekey', 'place_county', 'MSA_indicator', 'MSAnamelsad', 'is_top20', 'place_cbg', 'Type', 'Lat', 'Lon']]

In [13]:
df_place_geom = geopandas.GeoDataFrame(df_select[['Placekey', 'Lat', 'Lon']], geometry=geopandas.points_from_xy(df_select.Lon, df_select.Lat, crs="EPSG:4326"))
df_place_geom.set_index("Placekey", inplace=True)
df_place_geom.to_crs(3857, inplace=True)

In [15]:
df_cbg_msa = pd.read_csv(folder+'/SelectedVariableandCentroid_CBG.csv')

In [16]:
df_cbg_msa = df_cbg_msa[['CBGcode', 'MetroName']]
df_cbg_msa = df_cbg_msa.set_index('CBGcode')
df_cbg_msa.rename(columns={'MetroName':'source_msa'}, inplace=True)

In [17]:
df_od = dd.read_csv(folder+'/modified_cbg_monthly_od_full.csv')

In [ ]:
df_od.head()

In [19]:
df_od['date'] = df_od['year'].astype(str) + '-'+df_od['month'].astype(str).str.zfill(2)

In [ ]:
df_od.head()

In [21]:
df_join = df_od.join(df_select.set_index('Placekey'), on='placekey')

In [23]:
df_join = df_join.join(df_cbg_msa, on='cbg')

In [24]:
# https://stackoverflow.com/questions/67426361/adding-new-dask-column-based-on-a-vectorized-function
# or try this
def myfunc(df):
    df['self_loop'] = da.where(df['cbg']==df['place_cbg'], 1, 0)
    df['self_loop_county'] = da.where(df['county']==df['place_county'], 1, 0)
    df['place_cbg_str'] = df['place_cbg'].astype('Int64').astype(str).str.zfill(12)
    df['cbg_str'] = df['cbg'].astype(str).str.zfill(12)
    return df

df_join = df_join.map_partitions(myfunc)

In [ ]:
gdf = geopandas.read_file(shp_path)

In [26]:
gdf.set_index("GEOID", inplace=True)
gdf.to_crs(3857, inplace=True)

In [27]:
def myfuncDistance(df):
    df['distance'] = df_place_geom.loc[df['placekey']].geometry.reset_index(drop=True).distance(gdf.loc[df['cbg_str']].geometry.centroid.reset_index(drop=True), align=False).to_numpy()
    return df

meta = dd.utils.make_meta(df_join)
meta['distance'] = 0.0
df_join = df_join.map_partitions(myfuncDistance, meta=meta)

In [28]:
df_join.to_parquet(output_tmp_path)

(None,)

In [29]:
df_join = dd.read_parquet(output_tmp_path)

# Only select rows where source and target are in the same MSA

In [30]:
df_join = df_join.loc[df_join['MSAnamelsad'] == df_join['source_msa']]

In [31]:
df_join.to_parquet(output_tmp_msa_only_path)

(None,)

# calculate categories

In [32]:
df_join = dd.read_parquet(output_tmp_msa_only_path)

In [34]:
bmin, bmax = dd.compute(df_join['distance'].min(), df_join['distance'].max())
bins_100 = np.linspace(bmin, bmax, num=101) # 101 points, 100 categories

bins_100_mid = []
for i in range(len(bins_100)-1):
    bins_100_mid.append((bins_100[i]+bins_100[i+1])/2)

In [37]:
# setting the first element to 0, easier for filtering at the later stage
bins_100_mid[0]=0

In [38]:
# don't do cut within groups, faster and also unified range
# https://issues.apache.org/jira/browse/ARROW-14569?page=com.atlassian.jira.plugin.system.issuetabpanels%3Aall-tabpanel pyarrow has a issue that has to specify the labels for now
# for dask https://stackoverflow.com/questions/67790368/will-dask-map-partitionspd-cut-bins-actually-operate-on-entire-dataframe, has to pass the bins as an array
df_join = df_join.assign(distance_bin_100_cat = df_join['distance'].map_partitions(pd.cut, bins=bins_100, include_lowest=True, labels = bins_100_mid))

def convert_cat(df):
    return df.to_numpy()

df_join = df_join.assign(distance_bin_100 = df_join['distance_bin_100_cat'].map_partitions(convert_cat))

In [39]:
df_join.to_parquet(output_path)

(None,)